In [1]:
import keras 
from keras.models import load_model

# Load the Keras model from the .h5 file
model = keras.models.load_model('D:/Sign Detection/test/.h5')

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import math
from tensorflow import keras

# Initialize MediaPipe Hands model
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    static_image_mode=True,
    max_num_hands=1,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5)
offset = 20
imgSize = 28
image=cv2.imread('american_sign_language.PNG')
# Load your trained model
model = keras.models.load_model('D:/Sign Detection/test/keras_model.h5')  # Adjust the path as per your model file location

# Define function to calculate bounding box
def bbx(hand_landmarks, frame_width, frame_height):
    x_min, x_max = frame_width, 0
    y_min, y_max = frame_height, 0

    for landmark in hand_landmarks.landmark:
        x, y = int(landmark.x * frame_width), int(landmark.y * frame_height)
        if x < x_min:
            x_min = x
        if x > x_max:
            x_max = x
        if y < y_min:
            y_min = y
        if y > y_max:
            y_max = y

    return x_min, y_min, x_max - x_min, y_max - y_min

# Define the alphabet labels
alphabet_labels = {
    0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E', 5: 'F', 6: 'G', 7: 'H', 8: 'I', 9: 'J',
    10: 'K', 11: 'L', 12: 'M', 13: 'N', 14: 'O', 15: 'P', 16: 'Q', 17: 'R', 18: 'S', 19: 'T',
    20: 'U', 21: 'V', 22: 'W', 23: 'X', 24: 'Y', 25: 'Z'
}

# Initialize OpenCV VideoCapture
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()

    frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
    imgOutput = frame.copy()
    frame_height, frame_width, _ = frame.shape 

    # Process frame with MediaPipe Hands
    results = hands.process(frame)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            x, y, w, h = bbx(hand_landmarks, frame_width, frame_height)
            imgWhite = np.ones((imgSize, imgSize, 3), np.uint8) * 255
            imgCrop = frame[y - offset:y + h + offset, x - offset:x + w + offset]

            # Resize and preprocess the cropped image if needed
            aspectRatio = h / w
            if aspectRatio > 1:
                k = imgSize / h
                wCal = math.ceil(k * w)
                imgResize = cv2.resize(imgCrop, (wCal, imgSize))
                wGap = math.ceil((imgSize - wCal) / 2)
                imgWhite[:, wGap:wCal + wGap] = imgResize
            else:
                k = imgSize / w
                hCal = math.ceil(k * h)
                imgResize = cv2.resize(imgCrop, (imgSize, hCal))
                hGap = math.ceil((imgSize - hCal) / 2)
                imgWhite[hGap:hCal + hGap, :] = imgResize
    
            #cv2.imshow("ImageCrop", imgCrop)
            #cv2.imshow("ImageWhite", imgWhite)
            imgWhite = np.expand_dims(imgWhite, axis=0)
            prediction = model.predict(imgWhite)
            prediction = np.argmax(prediction)

            # Get the corresponding alphabet label for the prediction
            gesture_label = alphabet_labels.get(prediction, 'Unknown')

            # Draw rectangle around the hand region
            cv2.rectangle(imgOutput, (x - offset, y - offset - 50),
                          (x - offset + 90, y - offset - 50 + 50), (255, 0, 255), cv2.FILLED)

            # Put the predicted alphabet label on the image
            cv2.putText(imgOutput, gesture_label, (x, y - 26), cv2.FONT_HERSHEY_COMPLEX, 1.7, (255, 255, 255), 2, cv2.LINE_AA)

            cv2.rectangle(imgOutput, (x - offset, y - offset),
                          (x + w + offset, y + h + offset), (255, 0, 255), 4)

    # Display the hand prediction output
    cv2.imshow("Hand Prediction", imgOutput)
    cv2.imshow('american_sign_language',image)

    # Check for 'q' key to exit the loop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the VideoCapture and close all windows
cap.release()
cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 345ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
